#Food-101 olab
![picture](https://drive.google.com/uc?id=1Ar6KpqlsLxcLlnOrXPx6QZejStP6bV9P)

## Pre requisitos

Se actualiza fastai descargando y ejecutando el script *colab*

In [0]:
 !curl -s https://course.fast.ai/setup/colab | bash

Dar permiso para acceder a mi Drive

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#gdrive_dir = "/content/gdrive/My Drive/"
#curso_dir = gdrive_dir + 'Colab Notebooks/Curso fastai/'

Nos aseguramos que cualquier cambio en una librería es recargado automáticamente y que que cualquier gráfica o imagen se muestre aquí

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Food-101

Importamos los módulos de las librerías que necesitamos

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

Establecemos un tamaño de *batch size*, si obtenemos un error de *out of memory* reiniciar el kernel y usar un tamaño de *batch size* menor

In [0]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

### Descarga, preparación y visualización de los datos

In [0]:
path = untar_data(URLs.FOOD); path

In [0]:
path.ls()

In [0]:
path_images = path/'images'
path_images.ls()

In [0]:
np.random.seed(2)

In [0]:
data = ImageDataBunch.from_folder(path_images, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=224, num_workers=16).normalize(imagenet_stats)

In [0]:
data.show_batch(rows=4, figsize=(10,10))

In [0]:
print(data.classes)
len(data.classes),data.c

**¡Hay 101 clases!**

In [0]:
%cd /root/.fastai/data/food-101/images
!mkdir sub_images
!mv /root/.fastai/data/food-101/images/spaghetti_carbonara/ /root/.fastai/data/food-101/images/sub_images
!mv /root/.fastai/data/food-101/images/fried_rice /root/.fastai/data/food-101/images/sub_images
!mv /root/.fastai/data/food-101/images/miso_soup /root/.fastai/data/food-101/images/sub_images

In [0]:
!ls /content/data/food-101/images/sub_images

In [0]:
path_subimages = path_images/'sub_images'
path_subimages.ls()

In [0]:
subdata = ImageDataBunch.from_folder(path_subimages, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=224, num_workers=16).normalize(imagenet_stats)
subdata.show_batch(rows=4, figsize=(10,10))

In [0]:
print(subdata.classes)
len(subdata.classes), subdata.c

### Training: resnet34

In [0]:
learn = cnn_learner(subdata, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')

### Resultados

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(subdata.valid_ds)==len(losses)==len(idxs)

In [0]:
interp.plot_top_losses(9, figsize=(15,15))

In [0]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [0]:
interp.most_confused(min_val=2)

### Unfreezing, fine-tuning, y learning rates

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(6e-7,1e-6))

In [0]:
learn.save('stage-2')